<a href="https://colab.research.google.com/github/DataSaintist/GenAI/blob/main/Finetuning_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
from datasets import load_dataset

In [2]:
load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
squad = load_dataset("squad")

In [4]:
squad["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [5]:
squad["train"][:1]

{'id': ['5733be284776f41900661182'],
 'title': ['University_of_Notre_Dame'],
 'context': ['Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'],
 'question': ['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'],
 'answers': [{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}]}

In [6]:
squad["validation"][:1]

{'id': ['56be4db0acb8001400a502ec'],
 'title': ['Super_Bowl_50'],
 'context': ['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'],
 'question': ['Which NFL team represented the AFC at Super Bowl 50?'],
 'answers': [{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Br

In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [11]:
text = "Which NFL team represented the AFC at Super Bowl 50?"

In [12]:
context="Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the 'golden anniversary' with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as 'Super Bowl L'), so that the logo could prominently feature the Arabic numerals 50."

In [13]:
input = tokenizer(text, context, return_tensors="pt")

In [14]:
input

{'input_ids': tensor([[  101,  2029,  5088,  2136,  3421,  1996, 10511,  2012,  3565,  4605,
          2753,  1029,   102,  3565,  4605,  2753,  2001,  2019,  2137,  2374,
          2208,  2000,  5646,  1996,  3410,  1997,  1996,  2120,  2374,  2223,
          1006,  5088,  1007,  2005,  1996,  2325,  2161,  1012,  1996,  2137,
          2374,  3034,  1006, 10511,  1007,  3410,  7573, 14169,  3249,  1996,
          2120,  2374,  3034,  1006, 22309,  1007,  3410,  3792, 12915,  2484,
          1516,  2184,  2000,  7796,  2037,  2353,  3565,  4605,  2516,  1012,
          1996,  2208,  2001,  2209,  2006,  2337,  1021,  1010,  2355,  1010,
          2012, 11902,  1005,  1055,  3346,  1999,  1996,  2624,  3799,  3016,
          2181,  2012,  4203, 10254,  1010,  2662,  1012,  2004,  2023,  2001,
          1996, 12951,  3565,  4605,  1010,  1996,  2223, 13155,  1996,  1005,
          3585,  5315,  1005,  2007,  2536,  2751,  1011, 11773, 11107,  1010,
          2004,  2092,  2004,  8184, 2

In [15]:
attention_mask=input['attention_mask']
input_ids=input["input_ids"]

In [16]:
input_ids

tensor([[  101,  2029,  5088,  2136,  3421,  1996, 10511,  2012,  3565,  4605,
          2753,  1029,   102,  3565,  4605,  2753,  2001,  2019,  2137,  2374,
          2208,  2000,  5646,  1996,  3410,  1997,  1996,  2120,  2374,  2223,
          1006,  5088,  1007,  2005,  1996,  2325,  2161,  1012,  1996,  2137,
          2374,  3034,  1006, 10511,  1007,  3410,  7573, 14169,  3249,  1996,
          2120,  2374,  3034,  1006, 22309,  1007,  3410,  3792, 12915,  2484,
          1516,  2184,  2000,  7796,  2037,  2353,  3565,  4605,  2516,  1012,
          1996,  2208,  2001,  2209,  2006,  2337,  1021,  1010,  2355,  1010,
          2012, 11902,  1005,  1055,  3346,  1999,  1996,  2624,  3799,  3016,
          2181,  2012,  4203, 10254,  1010,  2662,  1012,  2004,  2023,  2001,
          1996, 12951,  3565,  4605,  1010,  1996,  2223, 13155,  1996,  1005,
          3585,  5315,  1005,  2007,  2536,  2751,  1011, 11773, 11107,  1010,
          2004,  2092,  2004,  8184, 28324,  2075,  

In [17]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])

In [18]:
import torch

In [19]:
with torch.no_grad():
  outputs = model(input_ids, attention_mask=attention_mask)

In [20]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 0.0734,  0.3875,  0.4131,  0.7448,  0.3007,  1.1624, -0.0361,  0.4046,
          0.1706,  0.7656,  0.3714,  0.0485,  0.4896,  0.3547,  0.8239,  0.4220,
          0.4867,  0.1682,  0.1690,  0.3288,  0.9251,  0.0141,  0.1727,  0.5325,
          0.8161,  0.6625,  1.1101,  0.0510,  0.2077,  0.5049,  0.8697,  0.1926,
          0.1359, -0.1576,  0.6642, -0.1841,  0.5446,  0.0821,  0.9923,  0.0662,
          0.2675,  0.0837,  0.9279,  0.0186,  0.0908,  0.8339,  0.3330,  0.0749,
          1.0232,  0.9866,  0.2713,  0.1767,  0.1960,  1.0397,  0.0298,  0.0739,
          0.7956,  0.4473,  0.3348,  0.4762,  0.7651,  0.6894,  0.7816,  0.5571,
          0.8001, -0.0620,  0.3455,  0.4999,  0.8678,  0.0806,  0.8341,  0.8257,
          0.7112,  0.5623,  0.3753,  0.0801,  0.2369, -0.1709, -0.0821, -0.0259,
          0.7687,  0.0165,  0.3218,  0.4704,  0.5951,  0.9922,  0.8775,  0.4450,
          0.2205,  0.6756,  0.3001,  0.7049,  0.2186, -0

In [21]:
outputs.start_logits

tensor([[ 0.0734,  0.3875,  0.4131,  0.7448,  0.3007,  1.1624, -0.0361,  0.4046,
          0.1706,  0.7656,  0.3714,  0.0485,  0.4896,  0.3547,  0.8239,  0.4220,
          0.4867,  0.1682,  0.1690,  0.3288,  0.9251,  0.0141,  0.1727,  0.5325,
          0.8161,  0.6625,  1.1101,  0.0510,  0.2077,  0.5049,  0.8697,  0.1926,
          0.1359, -0.1576,  0.6642, -0.1841,  0.5446,  0.0821,  0.9923,  0.0662,
          0.2675,  0.0837,  0.9279,  0.0186,  0.0908,  0.8339,  0.3330,  0.0749,
          1.0232,  0.9866,  0.2713,  0.1767,  0.1960,  1.0397,  0.0298,  0.0739,
          0.7956,  0.4473,  0.3348,  0.4762,  0.7651,  0.6894,  0.7816,  0.5571,
          0.8001, -0.0620,  0.3455,  0.4999,  0.8678,  0.0806,  0.8341,  0.8257,
          0.7112,  0.5623,  0.3753,  0.0801,  0.2369, -0.1709, -0.0821, -0.0259,
          0.7687,  0.0165,  0.3218,  0.4704,  0.5951,  0.9922,  0.8775,  0.4450,
          0.2205,  0.6756,  0.3001,  0.7049,  0.2186, -0.0454,  0.0936,  0.3901,
          0.0736, -0.1543,  

In [22]:
outputs.end_logits

tensor([[ 0.2382,  0.3176, -0.5117, -0.3841, -0.2401, -0.1363, -0.5772,  0.0318,
         -0.1991, -0.1566, -0.5683,  0.2543, -0.3315, -0.2233, -0.2058, -0.5418,
         -0.2255, -0.1838, -0.6991, -0.1752, -0.4113, -0.3525,  0.0667, -0.3527,
         -0.4017,  0.0095, -0.2344, -0.4535, -0.3417, -0.5586, -0.5747, -0.6081,
         -0.4432, -0.4273, -0.2523,  0.3571, -0.3125, -0.1257, -0.0952, -0.5349,
         -0.2814, -0.2348, -0.3949, -0.6121, -0.2471, -0.2316, -0.6533, -0.3898,
         -0.0638, -0.1145, -0.2479, -0.2939, -0.3217, -0.3825, -0.3976, -0.0937,
         -0.2196, -0.6634, -0.1691, -0.4355,  0.1188, -0.2940, -0.1383,  0.0622,
         -0.1278, -0.4121, -0.2263, -0.5046, -0.1251, -0.1262, -0.1102, -0.2892,
         -0.1040, -0.0245, -0.3584,  0.0995,  0.0687, -0.2686,  0.3102, -0.0383,
         -0.2825, -0.2563,  0.1329, -0.0368, -0.2918, -0.0525, -0.0510, -0.1901,
          0.0584, -0.0865,  0.0959, -0.1073,  0.0500,  0.2416, -0.1308,  0.2955,
         -0.1256, -0.2946,  

In [23]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 0.0734,  0.3875,  0.4131,  0.7448,  0.3007,  1.1624, -0.0361,  0.4046,
          0.1706,  0.7656,  0.3714,  0.0485,  0.4896,  0.3547,  0.8239,  0.4220,
          0.4867,  0.1682,  0.1690,  0.3288,  0.9251,  0.0141,  0.1727,  0.5325,
          0.8161,  0.6625,  1.1101,  0.0510,  0.2077,  0.5049,  0.8697,  0.1926,
          0.1359, -0.1576,  0.6642, -0.1841,  0.5446,  0.0821,  0.9923,  0.0662,
          0.2675,  0.0837,  0.9279,  0.0186,  0.0908,  0.8339,  0.3330,  0.0749,
          1.0232,  0.9866,  0.2713,  0.1767,  0.1960,  1.0397,  0.0298,  0.0739,
          0.7956,  0.4473,  0.3348,  0.4762,  0.7651,  0.6894,  0.7816,  0.5571,
          0.8001, -0.0620,  0.3455,  0.4999,  0.8678,  0.0806,  0.8341,  0.8257,
          0.7112,  0.5623,  0.3753,  0.0801,  0.2369, -0.1709, -0.0821, -0.0259,
          0.7687,  0.0165,  0.3218,  0.4704,  0.5951,  0.9922,  0.8775,  0.4450,
          0.2205,  0.6756,  0.3001,  0.7049,  0.2186, -0

In [24]:
start_scores=outputs.start_logits
end_scores=outputs.end_logits

In [25]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)+1

In [26]:
start_index

tensor(5)

In [27]:
end_index

tensor(146)

In [28]:
answer_tokens = input_ids[0][start_index:end_index]

In [29]:
tokenizer.decode(answer_tokens)

"the afc at super bowl 50? [SEP] super bowl 50 was an american football game to determine the champion of the national football league ( nfl ) for the 2015 season. the american football conference ( afc ) champion denver broncos defeated the national football conference ( nfc ) champion carolina panthers 24 – 10 to earn their third super bowl title. the game was played on february 7, 2016, at levi's stadium in the san francisco bay area at santa clara, california. as this was the 50th super bowl, the league emphasized the'golden anniversary'with various gold - themed initiatives, as well as temporarily suspending the tradition of naming each super bowl game with roman numerals ( under which the game would have"

In [30]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context.
    # This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [31]:
example=squad["validation"][:1]

In [32]:
prepare_train_features(example)

{'input_ids': [[101, 2029, 5088, 2136, 3421, 1996, 10511, 2012, 3565, 4605, 2753, 1029, 102, 3565, 4605, 2753, 2001, 2019, 2137, 2374, 2208, 2000, 5646, 1996, 3410, 1997, 1996, 2120, 2374, 2223, 1006, 5088, 1007, 2005, 1996, 2325, 2161, 1012, 1996, 2137, 2374, 3034, 1006, 10511, 1007, 3410, 7573, 14169, 3249, 1996, 2120, 2374, 3034, 1006, 22309, 1007, 3410, 3792, 12915, 2484, 1516, 2184, 2000, 7796, 2037, 2353, 3565, 4605, 2516, 1012, 1996, 2208, 2001, 2209, 2006, 2337, 1021, 1010, 2355, 1010, 2012, 11902, 1005, 1055, 3346, 1999, 1996, 2624, 3799, 3016, 2181, 2012, 4203, 10254, 1010, 2662, 1012, 2004, 2023, 2001, 1996, 12951, 3565, 4605, 1010, 1996, 2223, 13155, 1996, 1000, 3585, 5315, 1000, 2007, 2536, 2751, 1011, 11773, 11107, 1010, 2004, 2092, 2004, 8184, 28324, 2075, 1996, 4535, 1997, 10324, 2169, 3565, 4605, 2208, 2007, 3142, 16371, 28990, 2015, 1006, 2104, 2029, 1996, 2208, 2052, 2031, 2042, 2124, 2004, 1000, 3565, 4605, 1048, 1000, 1007, 1010, 2061, 2008, 1996, 8154, 2071, 14500

In [33]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [34]:
tokenized_datasets =squad.map(prepare_train_features,batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [35]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 88524
})

In [36]:
tokenized_datasets["validation"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 10784
})

In [37]:
from transformers  import TrainingArguments,Trainer

In [38]:
args =TrainingArguments(
    output_dir="finetune-BERT-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
from transformers import DefaultDataCollator

In [40]:
data_collator = DefaultDataCollator()

In [41]:
trainer =Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=tokenized_datasets["train"].select(range(1000)),
    eval_dataset=tokenized_datasets["validation"].select(range(100)),
    data_collator=data_collator,

)

In [42]:
import torch

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.388734


TrainOutput(global_step=125, training_loss=4.396931640625, metrics={'train_runtime': 87.2761, 'train_samples_per_second': 11.458, 'train_steps_per_second': 1.432, 'total_flos': 195972567552000.0, 'train_loss': 4.396931640625, 'epoch': 1.0})

In [43]:
from transformers import BertModel

In [44]:
import torch
from transformers import AutoTokenizer, BertLMHeadModel

In [45]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = BertLMHeadModel.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [46]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [47]:
outputs = model(**inputs, labels=inputs["input_ids"])

In [48]:
outputs

CausalLMOutputWithCrossAttentions(loss=tensor(15.6525, grad_fn=<NllLossBackward0>), logits=tensor([[[ -7.8962,  -7.8105,  -7.7903,  ...,  -7.0694,  -7.1693,  -4.3590],
         [ -8.4461,  -8.4401,  -8.5044,  ...,  -8.0625,  -7.9909,  -5.7160],
         [-15.2953, -15.4727, -15.5865,  ..., -12.9857, -11.7038, -11.4293],
         ...,
         [-14.0628, -14.2535, -14.3645,  ..., -12.7151, -11.1621, -10.2317],
         [-10.6576, -10.7892, -11.0402,  ..., -10.3233, -10.1578,  -3.7721],
         [-11.3383, -11.4590, -11.1767,  ...,  -9.2152,  -9.5209,  -9.5571]]],
       grad_fn=<ViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [49]:
loss = outputs.loss

In [50]:
loss

tensor(15.6525, grad_fn=<NllLossBackward0>)

In [51]:
logits = outputs.logits

In [52]:
logits

tensor([[[ -7.8962,  -7.8105,  -7.7903,  ...,  -7.0694,  -7.1693,  -4.3590],
         [ -8.4461,  -8.4401,  -8.5044,  ...,  -8.0625,  -7.9909,  -5.7160],
         [-15.2953, -15.4727, -15.5865,  ..., -12.9857, -11.7038, -11.4293],
         ...,
         [-14.0628, -14.2535, -14.3645,  ..., -12.7151, -11.1621, -10.2317],
         [-10.6576, -10.7892, -11.0402,  ..., -10.3233, -10.1578,  -3.7721],
         [-11.3383, -11.4590, -11.1767,  ...,  -9.2152,  -9.5209,  -9.5571]]],
       grad_fn=<ViewBackward0>)